In [29]:
import plotly.express as px

import numpy as np

In [30]:
from neo4j import GraphDatabase, basic_auth

from neo4j_process.cypher_requests import get_all_communes_with_coords


In [31]:
import os

import psycopg2

import pandas as pd

# POSTGRES Variables

HOST_POSTGRES = 'localhost' #"172.17.0.1"

POSTGRES_PORT = "5433"

POSTGRES_DATABASE = "postgres"

POSTGRES_TABLE = "insee_value"

POSTGRES_USER = "postgres"

POSTGRES_PASSWORD = "my_pass"

# NEO4J Variables

HOST_NEO4J = "bolt://localhost:7687"

PASSWORD_NEO4J = "passwordneo4j"


In [32]:
# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host= HOST_POSTGRES,
    port=POSTGRES_PORT,
    database=POSTGRES_DATABASE,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD)

conn.autocommit = True

# Création d'un curseur
cur = conn.cursor()




DETAIL:  The database was created using collation version 2.31, but the operating system provides version 2.36.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE postgres REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [33]:
query = "SELECT * FROM weighted_mode_2; "

In [34]:
result = cur.execute(query)

data= cur.fetchall()

conn.commit()
conn.close()

In [35]:
df_value = pd.DataFrame(data, columns=["insee", "value"])

In [36]:

# --- NEO4J Connection --- #

driver = GraphDatabase.driver(
  HOST_NEO4J,
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

In [37]:
with driver.session() as session:
       
    result = session.execute_write(get_all_communes_with_coords)

driver.close()

df_communes = pd.DataFrame(result)

In [38]:
df_communes

,insee,x,y
0,62053,2.143174,50.582893
1,80284,3.012390,49.717236
2,31290,0.610023,42.876728
3,28400,1.514000,48.094594
4,37220,0.099943,47.193507
...,...,...,...
34455,3170,3.092677,46.506657
34456,14394,-0.162877,49.015889
34457,29115,-3.895241,48.302289
34458,49308,-0.496845,47.370808


In [39]:
df_join_score = df_value.merge(df_communes, how="left", on="insee")

In [40]:
df_join_score

,insee,value,x,y
0,51418,23408,3.956960,49.238491
1,15237,36299,3.158477,45.060652
2,1224,82375,5.205392,45.773942
3,33518,1092,-0.405316,44.720807
4,76446,6288,0.999936,49.459291
...,...,...,...,...
33619,81304,318,2.117010,44.093800
33620,59084,188,2.404284,50.691533
33621,29114,65,-3.671922,48.489084
33622,29275,78,-3.679163,48.432630


In [41]:
df_join_score["ln_value"] = df_join_score["value"].apply(lambda v : np.log(v))

In [42]:
fig = px.scatter_mapbox(df_join_score, lat="y", lon="x", mapbox_style="open-street-map", color_continuous_scale="reds", color="ln_value", hover_name="insee")
fig.show()

In [43]:
fig.write_html("./../data/viz/viz_weighted_mode_new_1M_draws.html")